In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.outliers_influence import OLSInfluence
from scipy.stats import jarque_bera
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%capture
%run "5. Convertir variables.ipynb"

In [ ]:
def Modelo_Lineal_Robusto(Variables_X, Variable_Y, Nombre_Variable, 
                                                   Criterio_Eliminacion='aic', 
                                                   Umbral_VIF=5.0,
                                                   Alpha_Significancia=0.05,
                                                   Detectar_Outliers=True,
                                                   Iteracion=1, 
                                                   Variables_Originales=None,
                                                   Historial_Proceso=None):

    """
    Implementa regresión lineal robusta con metodología estadística rigurosa
    que incluye eliminación secuencial de variables, detección de 
    multicolinealidad mediante VIF, identificación de outliers influyentes
    y selección de modelos basada en criterios de información múltiples.
    
    Esta función aplica un enfoque metodológicamente sólido para construir
    modelos de regresión robustos mediante un proceso iterativo que:
    
    1. DETECCIÓN DE OUTLIERS: Identifica observaciones influyentes usando
       Distancia de Cook, leverage y residuos studentizados externamente.
    
    2. MULTICOLINEALIDAD RIGUROSA: Detecta multicolinealidad usando Factor
       de Inflación de Varianza (VIF) para todas las variables, no solo
       variables dummy predefinidas.
    
    3. ELIMINACIÓN SECUENCIAL: Elimina una variable por iteración (la menos
       contribuyente) permitiendo que la significancia se reajuste naturalmente.
    
    4. CRITERIOS MÚLTIPLES: Usa AIC, BIC y p-valores combinados para decisiones
       de selección de variables más informadas.
    
    5. ERRORES ESTÁNDAR ROBUSTOS: Implementa corrección HC3 para 
       heterocedasticidad y autocorrelación.
    
    6. DIAGNÓSTICOS COMPLETOS: Incluye tests de normalidad, 
       heterocedasticidad y autocorrelación de residuos.
    
    Parámetros:
    -----------
    Variables_X : pandas.DataFrame
        DataFrame con variables independientes numéricas. Debe estar
        completamente limpio sin valores faltantes ni variables categóricas
        sin codificar.
    
    Variable_Y : pandas.Series
        Variable dependiente numérica continua del mismo tamaño que Variables_X.
        No debe contener valores faltantes o infinitos.
    
    Nombre_Variable : str
        Nombre descriptivo de la variable dependiente para reportes y
        documentación de resultados.
    
    Criterio_Eliminacion : str, opcional (default='aic')
        Criterio principal para eliminación de variables:
        - 'aic': Akaike Information Criterion (penaliza complejidad)
        - 'bic': Bayesian Information Criterion (más conservador)
        - 'pvalue': Solo significancia estadística tradicional
        - 'combinado': Usa AIC + p-valor simultáneamente
    
    Umbral_VIF : float, opcional (default=5.0)
        Umbral máximo para Factor de Inflación de Varianza.
        Variables con VIF > Umbral_VIF serán eliminadas por multicolinealidad.
        Valores típicos: 5.0 (moderado), 10.0 (liberal), 2.5 (conservador).
    
    Alpha_Significancia : float, opcional (default=0.05)
        Nivel de significancia para pruebas estadísticas y selección
        de variables. Típicamente 0.05, 0.01 o 0.10.
    
    Detectar_Outliers : bool, opcional (default=True)
        Si True, identifica y reporta observaciones influyentes usando
        Distancia de Cook > 4/n, leverage > 2p/n y residuos studentizados
        externos > 3. No las elimina automáticamente.
    
    Iteracion : int, opcional (default=1)
        Parámetro interno para recursión. No modificar manualmente.
    
    Variables_Originales : list, opcional (default=None)
        Lista interna de variables del modelo inicial. Se inicializa
        automáticamente en la primera iteración.
    
    Historial_Proceso : list, opcional (default=None)
        Lista interna que registra el historial de eliminaciones y
        métricas en cada iteración para análisis post-hoc.
    
    Retorna:
    --------
    dict
        Diccionario completo con resultados del modelo final robusto:
        
        INFORMACIÓN BÁSICA:
        - 'Variable_Dependiente': Nombre de la variable dependiente
        - 'N_Observaciones': Número de observaciones utilizadas
        - 'N_Observaciones_Outliers': Observaciones identificadas como influyentes
        
        MÉTRICAS DE AJUSTE:
        - 'R_Cuadrado': Coeficiente de determinación R²
        - 'R_Cuadrado_Ajustado': R² ajustado por grados de libertad
        - 'AIC': Akaike Information Criterion
        - 'BIC': Bayesian Information Criterion
        - 'F_Estadistico': Estadístico F global del modelo
        - 'P_Valor_Modelo': P-valor del test F global
        
        VARIABLES Y COEFICIENTES:
        - 'Variables_Significativas': Lista de predictores finales
        - 'Variables_Eliminadas_Total': Variables removidas del modelo inicial
        - 'Coeficientes': Diccionario con coeficientes estimados
        - 'P_Valores': P-valores de cada coeficiente
        - 'Intervalos_Confianza_95': Intervalos de confianza robustos
        - 'VIF_Final': Factores de inflación de varianza finales
        
        PROCESO Y DIAGNÓSTICOS:
        - 'Iteraciones_Totales': Número de iteraciones realizadas
        - 'Criterio_Utilizado': Criterio de selección empleado
        - 'Outliers_Detectados': Índices de observaciones influyentes
        - 'Historial_Eliminaciones': Registro completo del proceso iterativo
        
        TESTS DIAGNÓSTICOS:
        - 'Test_Normalidad_JB': Jarque-Bera test (estadístico, p-valor)
        - 'Test_Heterocedasticidad_BP': Breusch-Pagan test (estadístico, p-valor)
        - 'Durbin_Watson': Test de autocorrelación
        - 'Numero_Condicion': Número de condición de la matriz X'X
        
        OBJETOS TÉCNICOS:
        - 'Modelo_Objeto': Objeto statsmodels fitted model
        - 'Matriz_Covarianza_Robusta': Matriz de covarianza HC3
    
    Ejemplo de uso:
    ---------------
    >>> # Modelo robusto con criterio AIC y detección de outliers
    >>> Resultado = Modelo_Lineal_Robusto(
    ...     Variables_X=df[['Edad', 'Educacion', 'Experiencia', 'Ingresos']], 
    ...     Variable_Y=df['Satisfaccion'],
    ...     Nombre_Variable='Satisfaccion_Laboral',
    ...     Criterio_Eliminacion='aic',
    ...     Umbral_VIF=5.0,
    ...     Detectar_Outliers=True
    ... )
    
    >>> # Examinar resultados clave
    >>> print(f"Variables finales: {Resultado['Variables_Significativas']}")
    >>> print(f"R² ajustado: {Resultado['R_Cuadrado_Ajustado']:.4f}")
    >>> print(f"AIC: {Resultado['AIC']:.2f}")
    >>> print(f"Outliers detectados: {len(Resultado['Outliers_Detectados'])}")
    
    >>> # Acceso a diagnósticos detallados
    >>> if Resultado['Test_Normalidad_JB'][1] < 0.05:
    ...     print("⚠️ Violación del supuesto de normalidad")
    >>> if any(vif > 5 for vif in Resultado['VIF_Final'].values()):
    ...     print("⚠️ Posible multicolinealidad residual")
    
    Notas metodológicas:
    --------------------
    - ELIMINACIÓN SECUENCIAL: A diferencia de métodos que eliminan múltiples
      variables simultáneamente, este enfoque elimina una por vez, permitiendo
      que los efectos se reajusten naturalmente en cada iteración.
    
    - ROBUSTEZ ESTADÍSTICA: Los errores estándar HC3 son robustos a
      heterocedasticidad y proporcionan inferencia válida incluso cuando
      se violan supuestos de homocedasticidad.
    
    - SELECCIÓN DE MODELOS: AIC penaliza la complejidad del modelo balanceando
      bondad de ajuste con parsimonia. BIC es más conservador y tiende a
      seleccionar modelos más simples.
    
    - MULTICOLINEALIDAD: VIF > 5 indica multicolinealidad moderada, VIF > 10
      indica multicolinealidad severa. Variables con VIF alto son eliminadas
      antes de la selección principal.
    
    - OUTLIERS: Se detectan pero no se eliminan automáticamente para preservar
      la integridad de los datos. El usuario debe evaluar si removerlos basándose
      en contexto sustantivo.
    
    - VALIDEZ DE SUPUESTOS: Los tests diagnósticos permiten evaluar violaciones
      de supuestos fundamentales de regresión lineal (normalidad, 
      heterocedasticidad, independencia).
    
    """
    
    # Inicializar parámetros en primera iteración.
    if Variables_Originales is None:
        Variables_Originales = Variables_X.columns.tolist()
    
    if Historial_Proceso is None:
        Historial_Proceso = []
    
    print(f"\n{'='*25} ITERACIÓN {Iteracion} {'='*25}")
    print(f"MODELO ROBUSTO METODOLÓGICAMENTE SÓLIDO: {Nombre_Variable}")
    print("="*70)
    
    # PASO 1: Verificar datos de entrada.
    if len(Variables_X.columns) == 0:
        print("\n❌ ERROR: No quedan variables independientes")
        return None
    
    if len(Variables_X) != len(Variable_Y):
        print(f"\n❌ ERROR: Dimensiones inconsistentes X({len(Variables_X)}) vs Y({len(Variable_Y)})")
        return None
    
    print(f"Variables en modelo actual: {len(Variables_X.columns)}")
    print(f"Observaciones disponibles: {len(Variables_X)}")
    
    # PASO 2: Detección de outliers influyentes (solo informativo).
    Indices_Outliers = []
    if Detectar_Outliers and Iteracion == 1:
        print(f"\n🔍 DETECCIÓN DE OUTLIERS INFLUYENTES:")
        
        # Modelo preliminar para calcular influence measures.
        Variables_X_Temp = sm.add_constant(Variables_X)
        Modelo_Temp = sm.OLS(Variable_Y, Variables_X_Temp).fit()
        Influence = OLSInfluence(Modelo_Temp)
        
        # Criterios múltiples para outliers.
        Cooks_Distance = Influence.cooks_distance[0]
        Leverage = Influence.hat_matrix_diag
        Residuos_Studentizados = Influence.resid_studentized_external
        
        # Umbrales estadísticos estándar.
        N_Obs = len(Variables_X)
        P_Vars = len(Variables_X.columns) + 1  # +1 por constante
        Umbral_Cooks = 4.0 / N_Obs
        Umbral_Leverage = 2.0 * P_Vars / N_Obs
        Umbral_Residuos = 3.0
        
        # Identificar outliers por cada criterio.
        Outliers_Cooks = np.where(Cooks_Distance > Umbral_Cooks)[0]
        Outliers_Leverage = np.where(Leverage > Umbral_Leverage)[0]
        Outliers_Residuos = np.where(np.abs(Residuos_Studentizados) > Umbral_Residuos)[0]
        
        # Unión de todos los criterios.
        Indices_Outliers = np.unique(np.concatenate([
            Outliers_Cooks, Outliers_Leverage, Outliers_Residuos
        ])).tolist()
        
        print(f"  Outliers por Distancia de Cook (>{Umbral_Cooks:.4f}): {len(Outliers_Cooks)}")
        print(f"  Outliers por Leverage (>{Umbral_Leverage:.4f}): {len(Outliers_Leverage)}")  
        print(f"  Outliers por Residuos (>|{Umbral_Residuos}|): {len(Outliers_Residuos)}")
        print(f"  Total outliers únicos detectados: {len(Indices_Outliers)}")
        
        if len(Indices_Outliers) > 0:
            print(f"  ⚠️  {len(Indices_Outliers)} observaciones influyentes detectadas")
            print("  ℹ️  No se eliminan automáticamente - evalúe manualmente")
    
    # PASO 3: Detección y eliminación de multicolinealidad con VIF.
    print(f"\n📊 DETECCIÓN DE MULTICOLINEALIDAD (VIF):")
    
    Variables_X_VIF = Variables_X.copy()
    Variables_Eliminadas_VIF = []
    
    # Calcular VIF iterativamente hasta que todos sean < umbral.
    while True:
        if len(Variables_X_VIF.columns) < 2:
            break
            
        # Calcular VIF para todas las variables actuales.
        Variables_X_Con_Constante = sm.add_constant(Variables_X_VIF)
        VIF_Valores = {}
        
        for i, Variable in enumerate(Variables_X_VIF.columns):
            try:
                VIF = variance_inflation_factor(Variables_X_Con_Constante.values, i + 1)
                VIF_Valores[Variable] = VIF
            except:
                VIF_Valores[Variable] = float('inf')
        
        # Encontrar variable con mayor VIF.
        Max_VIF_Variable = max(VIF_Valores.keys(), key=lambda x: VIF_Valores[x])
        Max_VIF_Valor = VIF_Valores[Max_VIF_Variable]
        
        if Max_VIF_Valor <= Umbral_VIF:
            break
        
        # Eliminar variable con mayor VIF.
        print(f"  Eliminando {Max_VIF_Variable} (VIF = {Max_VIF_Valor:.2f})")
        Variables_Eliminadas_VIF.append(Max_VIF_Variable)
        Variables_X_VIF = Variables_X_VIF.drop(columns=[Max_VIF_Variable])
    
    # Reportar VIFs finales.
    if len(Variables_X_VIF.columns) >= 2:
        Variables_X_Con_Constante = sm.add_constant(Variables_X_VIF)
        VIF_Final = {}
        for i, Variable in enumerate(Variables_X_VIF.columns):
            try:
                VIF_Final[Variable] = variance_inflation_factor(Variables_X_Con_Constante.values, i + 1)
            except:
                VIF_Final[Variable] = float('nan')
        
        Max_VIF_Final = max(VIF_Final.values()) if VIF_Final else 0
        print(f"  Variables restantes: {len(Variables_X_VIF.columns)}")
        print(f"  VIF máximo final: {Max_VIF_Final:.2f}")
        print(f"  Variables eliminadas por VIF: {len(Variables_Eliminadas_VIF)}")
    else:
        VIF_Final = {}
        print(f"  ⚠️  Muy pocas variables para calcular VIF")
    
    # Usar variables depuradas por VIF.
    Variables_X_Finales = Variables_X_VIF.copy()
    
    # PASO 4: Ajustar modelo con errores estándar robustos.
    if len(Variables_X_Finales.columns) == 0:
        print(f"\n❌ ERROR: Todas las variables eliminadas por multicolinealidad")
        return None
    
    print(f"\n🔧 AJUSTANDO MODELO ROBUSTO:")
    Variables_X_Con_Constante = sm.add_constant(Variables_X_Finales)
    Modelo = sm.OLS(Variable_Y, Variables_X_Con_Constante).fit(cov_type='HC3')
    
    # Métricas del modelo.
    print(f"  N observaciones: {Modelo.nobs:.0f}")
    print(f"  R²: {Modelo.rsquared:.4f}")
    print(f"  R² ajustado: {Modelo.rsquared_adj:.4f}")
    print(f"  AIC: {Modelo.aic:.2f}")
    print(f"  BIC: {Modelo.bic:.2f}")
    print(f"  F-estadístico: {Modelo.fvalue:.2f}")
    print(f"  P-valor modelo: {Modelo.f_pvalue:.6f}")
    
    # PASO 5: Evaluación de variables y criterios de eliminación.
    P_Valores = Modelo.pvalues
    Variables_Candidatas = [var for var in P_Valores.index if var != 'const']
    
    if len(Variables_Candidatas) == 0:
        print(f"\n❌ No hay variables predictoras en el modelo")
        return None
    
    # Identificar variable a eliminar según criterio.
    Variable_A_Eliminar = None
    
    if Criterio_Eliminacion == 'pvalue':
        # Criterio tradicional: eliminar la menos significativa si p >= alpha.
        P_Vals_Variables = {var: P_Valores[var] for var in Variables_Candidatas}
        Peor_Variable = max(P_Vals_Variables.keys(), key=lambda x: P_Vals_Variables[x])
        
        if P_Vals_Variables[Peor_Variable] >= Alpha_Significancia:
            Variable_A_Eliminar = Peor_Variable
            Razon_Eliminacion = f"p-valor = {P_Vals_Variables[Peor_Variable]:.6f} >= {Alpha_Significancia}"
    
    elif Criterio_Eliminacion in ['aic', 'bic']:
        # Criterio de información: evaluar mejora si eliminamos cada variable.
        AIC_Actual = Modelo.aic if Criterio_Eliminacion == 'aic' else Modelo.bic
        Mejor_Criterio = AIC_Actual
        
        for Variable_Test in Variables_Candidatas:
            # Probar modelo sin esta variable.
            Variables_Sin_Test = [v for v in Variables_Candidatas if v != Variable_Test]
            
            if len(Variables_Sin_Test) == 0:
                continue
                
            X_Test = sm.add_constant(Variables_X_Finales[Variables_Sin_Test])
            Modelo_Test = sm.OLS(Variable_Y, X_Test).fit(cov_type='HC3')
            Criterio_Test = Modelo_Test.aic if Criterio_Eliminacion == 'aic' else Modelo_Test.bic
            
            # Menor AIC/BIC es mejor.
            if Criterio_Test < Mejor_Criterio:
                Mejor_Criterio = Criterio_Test
                Variable_A_Eliminar = Variable_Test
                Razon_Eliminacion = f"{Criterio_Eliminacion.upper()} mejora: {AIC_Actual:.2f} → {Criterio_Test:.2f}"
    
    elif Criterio_Eliminacion == 'combinado':
        # Combinado: AIC + significancia.
        P_Vals_Variables = {var: P_Valores[var] for var in Variables_Candidatas}
        Variables_No_Sig = [var for var in Variables_Candidatas 
                           if P_Vals_Variables[var] >= Alpha_Significancia]
        
        if Variables_No_Sig:
            # Si hay no significativas, usar AIC entre ellas.
            AIC_Actual = Modelo.aic
            Mejor_AIC = AIC_Actual
            
            for Variable_Test in Variables_No_Sig:
                Variables_Sin_Test = [v for v in Variables_Candidatas if v != Variable_Test]
                X_Test = sm.add_constant(Variables_X_Finales[Variables_Sin_Test])
                Modelo_Test = sm.OLS(Variable_Y, X_Test).fit(cov_type='HC3')
                
                if Modelo_Test.aic < Mejor_AIC:
                    Mejor_AIC = Modelo_Test.aic
                    Variable_A_Eliminar = Variable_Test
                    Razon_Eliminacion = f"No significativa (p={P_Vals_Variables[Variable_Test]:.4f}) + AIC mejora"
    
    # PASO 6: Mostrar estado actual de variables.
    print(f"\n📋 ESTADO DE VARIABLES:")
    Variables_Significativas = [var for var in Variables_Candidatas 
                               if P_Valores[var] < Alpha_Significancia]
    Variables_No_Significativas = [var for var in Variables_Candidatas 
                                  if P_Valores[var] >= Alpha_Significancia]
    
    print(f"  Significativas (p < {Alpha_Significancia}): {len(Variables_Significativas)}")
    for Variable in Variables_Significativas:
        Beta = Modelo.params[Variable]
        P_Val = P_Valores[Variable]
        print(f"    ✅ {Variable:<30} β = {Beta:8.4f}, p = {P_Val:.6f}")
    
    print(f"  No significativas (p >= {Alpha_Significancia}): {len(Variables_No_Significativas)}")
    for Variable in Variables_No_Significativas:
        Beta = Modelo.params[Variable]  
        P_Val = P_Valores[Variable]
        print(f"    ❌ {Variable:<30} β = {Beta:8.4f}, p = {P_Val:.6f}")
    
    # Registrar iteración actual en historial.
    Historial_Actual = {
        'Iteracion': Iteracion,
        'Variables_Incluidas': Variables_Candidatas.copy(),
        'Variables_Significativas': Variables_Significativas.copy(),
        'R_Cuadrado_Ajustado': Modelo.rsquared_adj,
        'AIC': Modelo.aic,
        'BIC': Modelo.bic,
        'Variable_Eliminada': Variable_A_Eliminar,
        'Razon_Eliminacion': Razon_Eliminacion if Variable_A_Eliminar else None
    }
    Historial_Proceso.append(Historial_Actual)
    
    # PASO 7: Decidir siguiente acción.
    if Variable_A_Eliminar is None:
        # CASO BASE: Modelo final alcanzado.
        print(f"\n🎯 MODELO FINAL ALCANZADO en iteración {Iteracion}")
        print(f"Criterio '{Criterio_Eliminacion}' no sugiere más eliminaciones")
        
        # Ejecutar diagnósticos finales.
        print(f"\n🔬 DIAGNÓSTICOS FINALES:")
        
        # Test de normalidad Jarque-Bera.
        JB_Stat, JB_Pval = jarque_bera(Modelo.resid)
        print(f"  Jarque-Bera (normalidad): JB = {JB_Stat:.3f}, p = {JB_Pval:.6f}")
        
        # Test de heterocedasticidad Breusch-Pagan.
        from statsmodels.stats.diagnostic import het_breuschpagan
        BP_LM, BP_LM_Pval, BP_F, BP_F_Pval = het_breuschpagan(Modelo.resid, Variables_X_Con_Constante)
        print(f"  Breusch-Pagan (heterocedasticidad): LM = {BP_LM:.3f}, p = {BP_LM_Pval:.6f}")
        
        # Durbin-Watson (autocorrelación).
        from statsmodels.stats.stattools import durbin_watson
        DW_Stat = durbin_watson(Modelo.resid)
        print(f"  Durbin-Watson (autocorrelación): DW = {DW_Stat:.3f}")
        
        # Número de condición.
        Num_Condicion = np.linalg.cond(Variables_X_Con_Constante)
        print(f"  Número de condición: {Num_Condicion:.1f}")
        
        # Resumen del proceso completo.
        Variables_Eliminadas_Total = [var for var in Variables_Originales 
                                    if var not in Variables_Significativas]
        
        print(f"\n📊 RESUMEN PROCESO COMPLETO:")
        print(f"  Variables originales: {len(Variables_Originales)}")
        print(f"  Eliminadas por VIF: {len(Variables_Eliminadas_VIF)}")
        print(f"  Eliminadas por selección: {len(Variables_Eliminadas_Total) - len(Variables_Eliminadas_VIF)}")
        print(f"  Variables finales: {len(Variables_Significativas)}")
        print(f"  Iteraciones: {Iteracion}")
        print(f"  R² ajustado final: {Modelo.rsquared_adj:.4f}")
        print(f"  Outliers detectados: {len(Indices_Outliers)}")
        
        # Construir diccionario de resultados completo.
        Coeficientes = Modelo.params.to_dict()
        P_Valores_Dict = P_Valores.to_dict()
        Intervalos_Confianza = Modelo.conf_int().to_dict()
        
        Resultados_Finales = {
            # Información básica.
            'Variable_Dependiente': Nombre_Variable,
            'N_Observaciones': int(Modelo.nobs),
            'N_Observaciones_Outliers': len(Indices_Outliers),
            
            # Métricas de ajuste.
            'R_Cuadrado': Modelo.rsquared,
            'R_Cuadrado_Ajustado': Modelo.rsquared_adj,
            'AIC': Modelo.aic,
            'BIC': Modelo.bic,
            'F_Estadistico': Modelo.fvalue,
            'P_Valor_Modelo': Modelo.f_pvalue,
            
            # Variables y coeficientes.
            'Variables_Significativas': Variables_Significativas,
            'Variables_Eliminadas_Total': Variables_Eliminadas_Total,
            'Variables_Eliminadas_VIF': Variables_Eliminadas_VIF,
            'Coeficientes': Coeficientes,
            'P_Valores': P_Valores_Dict,
            'Intervalos_Confianza_95': Intervalos_Confianza,
            'VIF_Final': VIF_Final,
            
            # Proceso y metodología.
            'Iteraciones_Totales': Iteracion,
            'Criterio_Utilizado': Criterio_Eliminacion,
            'Umbral_VIF_Usado': Umbral_VIF,
            'Alpha_Significancia_Usado': Alpha_Significancia,
            'Outliers_Detectados': Indices_Outliers,
            'Historial_Eliminaciones': Historial_Proceso,
            
            # Tests diagnósticos.
            'Test_Normalidad_JB': (JB_Stat, JB_Pval),
            'Test_Heterocedasticidad_BP': (BP_LM, BP_LM_Pval),
            'Durbin_Watson': DW_Stat,
            'Numero_Condicion': Num_Condicion,
            
            # Objetos técnicos.
            'Modelo_Objeto': Modelo,
            'Matriz_Covarianza_Robusta': Modelo.cov_HC3
        }
        
        return Resultados_Finales
    
    else:
        # CASO RECURSIVO: Continuar eliminando variables.
        print(f"\n➡️  ELIMINANDO VARIABLE: {Variable_A_Eliminar}")
        print(f"   Razón: {Razon_Eliminacion}")
        
        Variables_X_Nuevas = Variables_X_Finales.drop(columns=[Variable_A_Eliminar])
        
        print(f"\nProcediendo a iteración {Iteracion + 1}...")
        
        return Modelo_Lineal_Robusto(
            Variables_X_Nuevas, Variable_Y, Nombre_Variable,
            Criterio_Eliminacion, Umbral_VIF, Alpha_Significancia,
            Detectar_Outliers, Iteracion + 1, Variables_Originales, 
            Historial_Proceso
        )